In [38]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
import textwrap as tr
import random
import os
import pinecone

In [39]:


# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [40]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./BankFAQs.csv", encoding='utf8')

data = loader.load()

In [41]:


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [42]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")




In [43]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Dot")

In [ ]:
'''index_name = "trial"

docsearch = Pinecone.from_documents(data_chunks, embeddings, index_name=index_name)'''

In [44]:
# make our prompt 
prompt_template = """Text: {context}

Question: {question}

Answer the question based on the text provided. If the text doesn't contain the answer, reply that the answer is not available."""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [45]:
question = "What do i do when I lost my credit card?"

In [46]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [47]:
answer['result']

' Call HDFC Bank 24x7 PhoneBanking immediately and report the loss of your card. The card can also be hotlisted using our Prepaid NetBanking facility In case you have opted for a back-up card, then you can request to activate back-up card and transfer balance on your hotlisted card to the back-up card. Balance transfer will be carried out instantaneously.'

In [48]:
def question_and_answer(question):
    qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

    answer = qa({"query": question})

    return answer['result']

In [50]:
question_and_answer("how do i create an a savings account")

' Answer: To open a Savings Account you have to: Submit your identity proof and address proof along with your latest passport size photographs Submit self-attested copies and originals for verification View more'